In [9]:
import numpy as np
import tkinter as tk
import random
from tkinter import messagebox

In [ ]:
class TicTacToe3D:
    # Contains all the game"s logic allowing for the GUI to interact with these methods to update state and display the boad
    def __init__(self, board_size=4):
        self.board_size = board_size
        # 3D list to represent the board: board[z][y][x]
        self.board = [[['' for _ in range(board_size)] for _ in range(board_size)] for _ in range(board_size)]
        self.players = ['X', 'O']
        self.current_player_index = 0  # 0 means 'X', 1 means 'O'


    # get_current_player and switch_player help make sure that players take turns alternatingly
    def get_current_player(self):
        return self.players[self.current_player_index]

    def switch_player(self):
        self.curent_player_index=1-self.current_player_index

    def is_valid_move(self, z, y, x):
        # Check if coordinates are within the board"s boundaries and is not clearly occupied
        if not (0 <= z < self.board_size and 0 <= y < self.board_size and 0 <= x < self.board_size):
            return False
        
        # Check if the chosen cell is empty
        if self.board[z][y][x] != "":
            return False
        
        # If all checks pass, the move is valid
        return True
    
    def make_move(self, z,y,x):
        # Allows player to make move assuming is_valid_move function passes
        if self.is_valid_move(z,y,x):
            self.board[z][y][x]=self.get_current_player()
            return True
        else:
            return False
        
    def undo_move(self, z, y, x):
        # Undo a move (used by AI when exploring moves)
        self.board[z][y][x] = ''

    def get_empty_cells(self):
        #Return a list of all empty positions (z, y, x) to help with alpha beta procedure
        empty_cells=[]
        for z in range(self.board_size):
            for y in range(self.board_size):
                for x in range(self_board_size):
                    if self.board[z][y][x]=='':
                        empty_cells.append((z,y,x))

    
    def check_win(self,z,y,x):
        # Checks for a winning scenario
        player=self.board(z,y,x)
        if not player:
            return False
        
        # Check all possible winning lines from last move
        winning_directions=self._get_winning_directions(z,y,x)
        for direction in winning_directions:
            if self._check_line(z,y,x,direction,player):
                return True
        return False
    

    def _get_winning_directions(self,z,y,x):
        directions=[]
        # Planes
        directions.extend([(0, 1, 0), (0, 0, 1), (0, 1, 1), (0, 1, -1)]) 

        # Columns
        directions.append((1, 0, 0)) # Vertical through the layers (changes in z)

        # Add diagonals involving the Z-Axis
        if y == x: 
            directions.append((1, 1, 0)) # Diagonal YX in Z planes
        if y == (self.board_size - 1 - x): 
            directions.append((1, -1, 0)) # Anti-diagonal YX in Z planes
        if z == y: 
            directions.append((1, 1, 0)) # Diagonal ZY in X planes
        if z == (self.board_size - 1 - y): 
            directions.append((-1, 1, 0)) # Anti-diagonal ZY in X planes
        if z == x: 
            directions.append((1, 0, 1)) # Diagonal ZX in Y planes
        if z == (self.board_size - 1 - x): 
            directions.append((-1, 0, 1)) # Anti-diagonal ZX in Y planes
        if z == y and y == x: 
            directions.append((1, 1, 1)) # Space diagonal
        if z == (self.board_size - 1 - y) and y == x: 
            directions.append((-1, 1, 1)) # Space anti-diagonal
        if z == (self.board_size - 1 - y) and y == (self.board_size - 1 - x): 
            directions.append((-1, 1, -1)) # Space anti-anti-diagonal
        if z == y and y == (self.board_size - 1 - x): 
            directions.append((1, 1, -1)) # Space anti-anti-diagonal
        return list(set(directions))

    def _check_line(self, z, y, x, direction, player):
        # Check and see if a winning line has been formed
        dz, dy, dx = direction
        count = 1
        # Check forward direction
        for i in range(1, self.board_size):
            nz, ny, nx = z + i * dz, y + i * dy, x + i * dx
            if 0 <= nz < self.board_size and 0 <= ny < self.board_size and 0 <= nx < self.board_size and self.board[nz][ny][nx] == player:
                count += 1
            else:
                break
        # Check backward direction
        for i in range(1, self.board_size):
            nz, ny, nx = z - i * dz, y - i * dy, x - i * dx
            if 0 <= nz < self.board_size and 0 <= ny < self.board_size and 0 <= nx < self.board_size and self.board[nz][ny][nx] == player:
                count += 1
            else:
                break
        return count >= self.board_size


    def is_board_full(self):
    # Check and see if the board is filled (DRAW)
        for z in range(self.board_size):
            for y in range(self.board_size):
                for x in range(self.board_size):
                    if self.board[z][y][x] == "":
                        return False
        return True

In [ ]:
class TicTacToeGUI:
    def __init__(self, master):
        """Initialize GUI elements and setup user interface."""
        self.master = master
        master.title("4x4x4 3D Tic-Tac-Toe")

        # Difficulty settings and AI depth lookup
        self.difficulty = tk.StringVar(value="Easy")
        self.depth_lookup = {"Easy": 2, "Difficult": 4, "Insane": 6}

        # Game state variables
        self.human_player = None
        self.ai_player = None
        self.buttons = None
        self.in_game = False

        # Selecting Difficulty
        tk.Label(master, text="Select Difficulty:").pack(pady=5)
        self.difficulty_buttons = []
        for mode in ["Easy", "Difficult", "Insane"]:
            rb = tk.Radiobutton(master, text=mode, variable=self.difficulty, value=mode)
            rb.pack(anchor=tk.W)
            self.difficulty_buttons.append(rb)

        # Button to start the game
        self.start_button = tk.Button(master, text="Start Game", command=self.start_game)
        self.start_button.pack(pady=10)

        # Status text
        self.status_label = tk.Label(master, text="", font=('Helvetica', 14))
        self.status_label.pack(pady=10)

        # Frame for the 3D board
        self.board_frame = tk.Frame(master)
        self.board_frame.pack()

        # Restart button
        self.restart_button = tk.Button(master, text="Restart Game", command=self.reset_ui)
        self.restart_button.pack(pady=10)

    # Set up Game
    def start_game(self):
        #Initialize a new game with the selected difficulty.
        if self.in_game:
            return  # Prevent starting multiple games at once
        self.in_game = True

        self.game = TicTacToe3D()
        # Set search depth based on difficulty
        self.depth = self.depth_lookup.get(self.difficulty.get(), 2)

        # Randomly decide which side (X or O) the human gets
        if random.choice([True, False]):
            self.human_player, self.ai_player = 'X', 'O'
        else:
            self.human_player, self.ai_player = 'O', 'X'

        # Create the interactive board buttons
        self.buttons = [[[None for _ in range(4)] for _ in range(4)] for _ in range(4)]
        self.create_board_ui()
        self.status_label.config(text=f"You are {self.human_player}. Your turn.")

        # Disable difficulty selection while game is active
        for rb in self.difficulty_buttons:
            rb.config(state=tk.DISABLED)

        # If AI starts the game
        if self.game.get_current_player() == self.ai_player:
            self.status_label.config(text="AI is thinking...")
            self.master.after(500, self.ai_move)


    # Creating the UI for the board
    def create_board_ui(self):
        # Create and display the 3D Grid of buttons
        # Remove any old layers
        for widget in self.board_frame.winfo_children():
            widget.destroy()

        # Each z-layer is a labeled frame
        for z in range(self.game.board_size):
            layer_frame = tk.LabelFrame(self.board_frame, text=f"Layer {z}", padx=5, pady=5)
            layer_frame.grid(row=0, column=z, padx=10)
            for y in range(self.game.board_size):
                for x in range(self.game.board_size):
                    button = tk.Button(
                        layer_frame, text='', width=4, height=2,
                        command=lambda z=z, y=y, x=x: self.handle_click(z, y, x)
                    )
                    button.grid(row=y, column=x, padx=2, pady=2)
                    self.buttons[z][y][x] = button


    # The code below helps with AI and Human moves
    def handle_click(self, z, y, x):
        """Handle the human player's move."""
        if not self.in_game or self.game.get_current_player() != self.human_player:
            return
        if self.game.make_move(z, y, x):
            self.update_ui(z, y, x)
            # Check if player won or game ended
            if self.check_game_end(z, y, x):
                return
            # Otherwise, AI takes a turn
            self.game.switch_player()
            self.status_label.config(text="AI is thinking...")
            self.master.after(500, self.ai_move)
    
    def ai_move(self):
        # Perform AI's move with alpha-pruning


    def alpha_beta(self, depth, alpha, beta, maximizing):
        # A recursive function for deciding best AI Move




    # Code to help with the game flow (updating the UI, checking if the current move ended the game, disable board buttons, end the game and renable difficulty options, and reset ui)

IndentationError: expected an indented block after function definition on line 109 (2510478686.py, line 113)

In [13]:
# Code to Run Tic Tac Toe Game
if __name__ == "__main__":
    root = tk.Tk()
    gui = TicTacToe_GUI(root)
    root.mainloop()

NameError: name 'board_sizes' is not defined